# Redox Buffering

With reference to deLevie, Redox Buffer Strength, J. Chem. Educ. 1999 https://doi.org/10.1021/ed076p574, which draws an explicit parallel between pH buffering and redox buffering, albeit not exploring the dynamics.

In [1]:
import projectpath

import equilibrator_api
from equilibrator_api.component_contribution import FARADAY, R, Q_
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy

from mosmo.knowledge import kb
from mosmo.model import DS, DbXref, KbEntry, Molecule, Reaction, ReactionNetwork
from mosmo.calc import convenience_kinetics

jax.config.update('jax_enable_x64', True)

KB = kb.configure_kb()

cc = equilibrator_api.ComponentContribution()
cc.p_h = Q_(7.3)
cc.p_mg = Q_(1.5)
cc.ionic_strength = Q_("0.25M")
cc.temperature = Q_("298.15K")
RT = cc.RT

def find_cc_met(met):
    for xref in (met.xrefs or []):
        if xref.db == DS.KEGG:
            return cc.get_compound(f'KEGG:{xref.id}')
    return None

def delta_g(reaction):
    cc_mets = {met: (find_cc_met(met), count) for met, count in reaction.stoichiometry.items()}
    cc_rxn = equilibrator_api.Reaction({cc_met: count for cc_met, count in cc_mets.values()})
    return cc.standard_dg_prime(cc_rxn), {met: cc.standard_dg_formation(cc_met)[0] for met, (cc_met, count) in cc_mets.items()}

## Transhydrogenase Dynamics; Equilibrium between NAD(H) and NADP(H)

In [2]:
nad = KB('nad.ox')
nadh = KB('nad.red')
nadp = KB('nadp.ox')
nadph = KB('nadp.red')
proton = KB('h+')

# All concentrations in mM
concs = {
    nad: 2.55,
    nadh: 8.36e-02,
    nadp: 2.08e-03,
    nadph: 1.21e-01,
    proton: pow(10, -7.3) * 1000,
}

transhydrogenase = Reaction(
    id='TRANSHYD',
    name='NAD(P) Transhydrogenase',
    stoichiometry={
        nad: -1,
        nadph: -1,
        nadh: +1,
        nadp: +1,
    },
    xrefs=[DbXref('RHEA', '11692'), DbXref('EC', '1.6.1.1'), DbXref('KEGG', 'R00112')],
    catalyst=Molecule(id='sTH', name='Soluble Transhydrogenase'),
    reversible=True,
)

### ΔG, E, and Kinetic Parameters

- half-reaction potential E = ΔG / nF

In [3]:
def delta_g_report(reaction):
    dg_r, dgf_m = delta_g(reaction)
    print(f'[{reaction.id}] {reaction.name}: {reaction.formula}')
    print(f'ΔG° = {dg_r}')
    for met, count in reaction.stoichiometry.items():
        print(f'    {count:+2d} {dgf_m[met]:8.2f} [{met.id}] {met.name}')

delta_g_report(transhydrogenase)

dg_r, dgf_m = delta_g(transhydrogenase)
print()
print(f'reaction ΔG = ΔG° + RT ln(Q) = {dg_r + RT * sum(count * np.log(concs[mol] / 1000) for mol, count in transhydrogenase.stoichiometry.items())}')

print()
for ox, red in ((nad, nadh), (nadp, nadph)):
    print(f'[{ox.name} => {red.name}]:  E = {Q_(dgf_m[red] - dgf_m[ox], "kilojoule / mole") / (-2 * FARADAY)}')

[TRANSHYD] NAD(P) Transhydrogenase: NAD+ + NADPH <=> NADH + NADP+
ΔG° = (2.98 +/- 0.34) kilojoule / mole
    -1 -2261.38 [nad.ox] NAD+
    -1 -3070.53 [nadp.red] NADPH
    +1 -2236.55 [nad.red] NADH
    +1 -3098.93 [nadp.ox] NADP+

reaction ΔG = ΔG° + RT ln(Q) = (-15.56 +/- 0.34) kilojoule / mole

[NAD+ => NADH]:  E = -0.12866755389106713 kilojoule / kilocoulomb
[NADP+ => NADPH]:  E = -0.1472002933851714 kilojoule / kilocoulomb


**NOTE** The ΔG of formation values are way off from the website, though the reaction ΔG is consistent. Worth investigating, or a rabbit hole?
We'll want to nail all this down before publication or release, but it's not a blocker to development.